In [12]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

from skimage import io
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
spotify_data = pd.read_csv('SpotifyFeatures.csv')
spotify_data.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


In [14]:
spotify_features_df = spotify_data
genre_OHE = pd.get_dummies(spotify_features_df.genre)
key_OHE = pd.get_dummies(spotify_features_df.key)

In [15]:
scaled_features = MinMaxScaler().fit_transform([
  spotify_features_df['acousticness'].values,
  spotify_features_df['danceability'].values,
  spotify_features_df['duration_ms'].values,
  spotify_features_df['energy'].values,
  spotify_features_df['instrumentalness'].values,
  spotify_features_df['liveness'].values,
  spotify_features_df['loudness'].values,
  spotify_features_df['speechiness'].values,
  spotify_features_df['tempo'].values,
  spotify_features_df['valence'].values,
  ])

In [16]:
spotify_features_df[['acousticness','danceability','duration_ms','energy','instrumentalness','liveness','loudness','speechiness','tempo','valence']] = scaled_features.T

In [17]:
spotify_features_df = spotify_features_df.drop('genre',axis = 1)
spotify_features_df = spotify_features_df.drop('artist_name', axis = 1)
spotify_features_df = spotify_features_df.drop('track_name', axis = 1)
spotify_features_df = spotify_features_df.drop('popularity',axis = 1)
spotify_features_df = spotify_features_df.drop('key', axis = 1)
spotify_features_df = spotify_features_df.drop('mode', axis = 1)
spotify_features_df = spotify_features_df.drop('time_signature', axis = 1)

In [18]:
#Appending the OHE columns of the categorical features
spotify_features_df = spotify_features_df.join(genre_OHE)
spotify_features_df = spotify_features_df.join(key_OHE)

In [19]:
spotify_features_df.head()

,track_id,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,...,B,C,C#,D,D#,E,F,F#,G,G#
0,0BRjO6ga9RKCKjfDqeFgWV,0.000025,0.000022,1.0,0.000028,0.000018,0.000022,0.0,0.000019,0.001699,...,0,0,1,0,0,0,0,0,0,0
1,0BjC1NfoEOOusryehmNudP,0.000042,0.000045,1.0,0.000046,0.000040,0.000042,0.0,0.000041,0.001307,...,0,0,0,0,0,0,0,1,0,0
2,0CoSDzoNIKCRs124s9uTVy,0.000087,0.000085,1.0,0.000082,0.000082,0.000082,0.0,0.000082,0.000666,...,0,1,0,0,0,0,0,0,0,0
3,0Gc6TVm52BwZD07Ki6tIvf,0.000084,0.000081,1.0,0.000082,0.000080,0.000081,0.0,0.000080,0.001207,...,0,0,1,0,0,0,0,0,0,0
4,0IuslXpMROHdEPvSl1fTQK,0.000267,0.000260,1.0,0.000259,0.000257,0.000258,0.0,0.000256,0.001957,...,0,0,0,0,0,0,1,0,0,0


In [20]:
client_id='bf194ab962224a71a3c511ce44cf59c1'
client_secret='c8310ad8442b4d5281643f0a876f9c85'

In [42]:
import spotipy
import time
from IPython.core.display import clear_output
from spotipy import SpotifyClientCredentials, util

client_id='c1f283e1d0fe4a869b90011a5c81a804'
client_secret='587d984268b1499ab5b3b1ba1d2996ab'
redirect_uri='http://localhost:8881/callback'
username = '31d7r2ju3zqj3ulbu2oiywoqjkdm'
scope = 'playlist-modify-public'

#Credentials to access the Spotify Music Data
manager = SpotifyClientCredentials(client_id,client_secret)
spt = spotipy.Spotify(client_credentials_manager=manager)

#Credentials to access to  the Spotify User's Playlist, Favorite Songs, etc. 
token = util.prompt_for_user_token(username,scope,client_id,client_secret,redirect_uri) 
sp = spotipy.Spotify(auth=token)

playlist_dic = {}
playlist_cover_art = {}

for i in sp.current_user_playlists()['items']:
    playlist_dic[i['name']] = i['uri'].split(':')[2]
    playlist_cover_art[i['uri'].split(':')[2]] = i['images'][0]['url']

print(playlist_dic)

C:\Users\PC\AppData\Local\Temp\ipykernel_9748\1441619392.py:3: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import clear_output


{'Marpril': '3oSH1TvSZo2qZqK5i7Pxpp', 'sad boi hours': '07PqZVNH9Ga4L3zdSKEJzJ', 'lifesaver': '75b6qLtvKMCHhnza4fM30O', 'Espressoes': '7d9PaNj7kny2uTR9wnnTJM', 'Frequent': '4c2BK9SZcaVpGBphzEsj24', 'Bathroom': '3qGZTfMJwscvBJrvaSBbkV', 'Cure depression': '67VE0i0yAsH7KSSv3fg0sz'}


In [43]:
def generate_playlist_df(playlist_name, playlist_dic, spotify_data):
    
    playlist = pd.DataFrame()

    for i, j in enumerate(sp.playlist(playlist_dic[playlist_name])['tracks']['items']):
        playlist.loc[i, 'artist'] = j['track']['artists'][0]['name']
        playlist.loc[i, 'track_name'] = j['track']['name']
        playlist.loc[i, 'track_id'] = j['track']['id']
        playlist.loc[i, 'url'] = j['track']['album']['images'][1]['url']
        playlist.loc[i, 'date_added'] = j['added_at']

    playlist['date_added'] = pd.to_datetime(playlist['date_added'])  
    
    playlist = playlist[playlist['track_id'].isin(spotify_data['track_id'].values)].sort_values('date_added',ascending = False)

    return playlist
playlist_df = generate_playlist_df('sad boi hours', playlist_dic, spotify_data) 

In [44]:
playlist_df.head()

,artist,track_name,track_id,url,date_added
37,Lorde,Liability,6Kkt27YmFyIFrcX3QXFi2o,https://i.scdn.co/image/ab67616d00001e02f8553e...,2020-11-09 01:58:13+00:00
33,The 1975,Be My Mistake,18UsAG7SfOQ5sxJEdjAMH0,https://i.scdn.co/image/ab67616d00001e026c5820...,2020-11-09 01:58:04+00:00
21,Billie Eilish,idontwannabeyouanymore,41zXlQxzTi6cGAjpOXyLYH,https://i.scdn.co/image/ab67616d00001e02a9f6c0...,2020-08-31 07:01:02+00:00
19,Conan Gray,Lookalike,4K3gM7kzX4POAfrs71HomN,https://i.scdn.co/image/ab67616d00001e021da538...,2020-08-31 06:30:56+00:00
16,Owl City,Vanilla Twilight,0vCTQcxSGAgjHaiAsIANKn,https://i.scdn.co/image/ab67616d00001e02d4f77b...,2020-08-31 06:30:10+00:00


In [45]:
def generate_playlist_vector(spotify_features, playlist_df, weight_factor):
    
    spotify_features_playlist = spotify_features[spotify_features['track_id'].isin(playlist_df['track_id'].values)]
    spotify_features_playlist = spotify_features_playlist.merge(playlist_df[['track_id','date_added']], on = 'track_id', how = 'inner')
    
    spotify_features_nonplaylist = spotify_features[~spotify_features['track_id'].isin(playlist_df['track_id'].values)]
    
    playlist_feature_set = spotify_features_playlist.sort_values('date_added',ascending=False)
    
    
    most_recent_date = playlist_feature_set.iloc[0,-1]
    
    for ix, row in playlist_feature_set.iterrows():
        playlist_feature_set.loc[ix,'days_from_recent'] = int((most_recent_date.to_pydatetime() - row.iloc[-1].to_pydatetime()).days)
        
    
    playlist_feature_set['weight'] = playlist_feature_set['days_from_recent'].apply(lambda x: weight_factor ** (-x))
    
    playlist_feature_set_weighted = playlist_feature_set.copy()
    
    playlist_feature_set_weighted.update(playlist_feature_set_weighted.iloc[:,:-3].mul(playlist_feature_set_weighted.weight.astype(int),0))   
    
    playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :-3]
    

    
    return playlist_feature_set_weighted_final.sum(axis = 0), spotify_features_nonplaylist


In [46]:
playlist_vector, nonplaylist_df = generate_playlist_vector(spotify_features_df, playlist_df, 1.2)
print(playlist_vector.shape)
print(nonplaylist_df.head())

(50,)
                 track_id  acousticness  danceability  duration_ms    energy  \
0  0BRjO6ga9RKCKjfDqeFgWV      0.000025      0.000022          1.0  0.000028   
1  0BjC1NfoEOOusryehmNudP      0.000042      0.000045          1.0  0.000046   
2  0CoSDzoNIKCRs124s9uTVy      0.000087      0.000085          1.0  0.000082   
3  0Gc6TVm52BwZD07Ki6tIvf      0.000084      0.000081          1.0  0.000082   
4  0IuslXpMROHdEPvSl1fTQK      0.000267      0.000260          1.0  0.000259   

   instrumentalness  liveness  loudness  speechiness     tempo  ...  B  C  C#  \
0          0.000018  0.000022       0.0     0.000019  0.001699  ...  0  0   1   
1          0.000040  0.000042       0.0     0.000041  0.001307  ...  0  0   0   
2          0.000082  0.000082       0.0     0.000082  0.000666  ...  0  1   0   
3          0.000080  0.000081       0.0     0.000080  0.001207  ...  0  0   1   
4          0.000257  0.000258       0.0     0.000256  0.001957  ...  0  0   0   

   D  D#  E  F  F#  G  G# 

In [47]:
def generate_recommendation(spotify_data, playlist_vector, nonplaylist_df):

    non_playlist = spotify_data[spotify_data['track_id'].isin(nonplaylist_df['track_id'].values)]
    non_playlist['sim'] = cosine_similarity(nonplaylist_df.drop(['track_id'], axis = 1).values, playlist_vector.drop(labels = 'track_id').values.reshape(1, -1))[:,0]
    non_playlist_top15 = non_playlist.sort_values('sim',ascending = False).head(15)
    non_playlist_top15['url'] = non_playlist_top15['track_id'].apply(lambda x: sp.track(x)['album']['images'][1]['url'])
    
    return  non_playlist_top15

In [48]:
top15 = generate_recommendation(spotify_data, playlist_vector, nonplaylist_df)  
top15

C:\Users\PC\AppData\Local\Temp\ipykernel_9748\43927750.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_playlist['sim'] = cosine_similarity(nonplaylist_df.drop(['track_id'], axis = 1).values, playlist_vector.drop(labels = 'track_id').values.reshape(1, -1))[:,0]


,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,sim,url
151032,Pop,Norah Jones,Turn Me On,6FjAGZp7c0Z2uaL3eHkXsx,68,0.000076,0.000074,1.0,0.000072,0.000070,A#,0.000071,0.0,Major,0.000071,0.000620,3/4,0.000074,0.912871,https://i.scdn.co/image/ab67616d00001e02786281...
109302,Pop,Ed Sheeran,I See Fire,6JyHE2IeTdfdwk7T6nRj0L,66,0.000070,0.000070,1.0,0.000068,0.000068,A#,0.000068,0.0,Minor,0.000068,0.000574,4/4,0.000069,0.912871,https://i.scdn.co/image/ab67616d00001e0210ab36...
113248,Pop,Lennon Stella,Like Everybody Else - Acoustic,4cRVPfsnX1r3S3GzXRy2wt,65,0.000083,0.000081,1.0,0.000079,0.000078,A#,0.000079,0.0,Minor,0.000078,0.000571,4/4,0.000079,0.912871,https://i.scdn.co/image/ab67616d00001e02e00934...
151742,Pop,Noah Cyrus,Lately,3s4hlJ1MVdqAHfAkY2V98j,60,0.000066,0.000066,1.0,0.000065,0.000063,F#,0.000064,0.0,Major,0.000063,0.000578,4/4,0.000066,0.912871,https://i.scdn.co/image/ab67616d00001e02a7bd84...
152212,Pop,Regina Spektor,The Call,341863z079BT4EFPiSw9DP,62,0.000089,0.000087,1.0,0.000084,0.000084,A#,0.000084,0.0,Major,0.000084,0.000569,3/4,0.000085,0.912871,https://i.scdn.co/image/ab67616d00001e026b0a69...
113575,Pop,Corey Kilgannon,Macaroni Song,7mE8MPVQRjvi6T2F0lDYkL,63,0.000062,0.000061,1.0,0.000060,0.000059,A#,0.000059,0.0,Major,0.000059,0.000591,4/4,0.000060,0.912871,https://i.scdn.co/image/ab67616d00001e02cc7bdb...
151559,Pop,H.E.R.,Facts,4MzxCIHY6TB9amFIvWsO2V,59,0.000068,0.000068,1.0,0.000067,0.000065,A#,0.000066,0.0,Minor,0.000066,0.000629,4/4,0.000067,0.912871,https://i.scdn.co/image/ab67616d00001e02b5d9c8...
151499,Pop,The 1975,Surrounded By Heads And Bodies,4EGVeQKn8WyzCmCx5YTIc4,62,0.000058,0.000057,1.0,0.000056,0.000055,F#,0.000055,0.0,Major,0.000055,0.000597,4/4,0.000056,0.912871,https://i.scdn.co/image/ab67616d00001e026c5820...
111411,Pop,The Mayries,Rockabye - Acoustic Version,30LDuVfrePWbedYTc1mUCn,74,0.000067,0.000065,1.0,0.000063,0.000062,A#,0.000062,0.0,Major,0.000062,0.000549,4/4,0.000064,0.912871,https://i.scdn.co/image/ab67616d00001e0232959a...
109165,Pop,Drake,Lust For Life,1v24T2ug4TlssYZvI3aL4O,71,0.000064,0.000062,1.0,0.000061,0.000059,F#,0.000060,0.0,Major,0.000060,0.000555,4/4,0.000059,0.912871,https://i.scdn.co/image/ab67616d00001e0254ef13...


In [49]:
curate = top15[['artist_name', 'track_name']]
arrai = curate.to_numpy()
arrai

array([['Norah Jones', 'Turn Me On'],
       ['Ed Sheeran', 'I See Fire'],
       ['Lennon Stella', 'Like Everybody Else - Acoustic'],
       ['Noah Cyrus', 'Lately'],
       ['Regina Spektor', 'The Call'],
       ['Corey Kilgannon', 'Macaroni Song'],
       ['H.E.R.', 'Facts'],
       ['The 1975', 'Surrounded By Heads And Bodies'],
       ['The Mayries', 'Rockabye - Acoustic Version'],
       ['Drake', 'Lust For Life'],
       ['Simon & Garfunkel', 'The Sounds of Silence - Acoustic Version'],
       ['Lorde', 'Sober'],
       ['blackbear', 'Rly Real'],
       ['Cat Power', 'Sea of Love'],
       ['Joji', 'Will He']], dtype=object)

In [50]:
for index, row in top15.iterrows():
    print(row['artist_name'] + " - " + row['track_name'])

Norah Jones - Turn Me On
Ed Sheeran - I See Fire
Lennon Stella - Like Everybody Else - Acoustic
Noah Cyrus - Lately
Regina Spektor - The Call
Corey Kilgannon - Macaroni Song
H.E.R. - Facts
The 1975 - Surrounded By Heads And Bodies
The Mayries - Rockabye - Acoustic Version
Drake - Lust For Life
Simon & Garfunkel - The Sounds of Silence - Acoustic Version
Lorde - Sober
blackbear - Rly Real
Cat Power - Sea of Love
Joji - Will He


In [22]:
playlist_df = generate_playlist_df('Cure depression', playlist_dic, spotify_data) 
playlist_vector, nonplaylist_df = generate_playlist_vector(spotify_features_df, playlist_df, 1.2)
top15 = generate_recommendation(spotify_data, playlist_vector, nonplaylist_df)  

C:\Users\PC\AppData\Local\Temp\ipykernel_15948\43927750.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_playlist['sim'] = cosine_similarity(nonplaylist_df.drop(['track_id'], axis = 1).values, playlist_vector.drop(labels = 'track_id').values.reshape(1, -1))[:,0]


,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,sim,url
111827,Pop,Rex Orange County,Uno,5WvDO1ZKap15ehQeNRHJ9H,66,0.000062,0.000062,1.0,0.000060,0.000057,F,0.000058,0.0,Minor,0.000059,0.001011,4/4,0.000060,0.769919,https://i.scdn.co/image/ab67616d00001e02cee03c...
113371,Pop,Mac Miller,Missed Calls,6ET22Ri3FYXi5C8BBqHWcO,62,0.000062,0.000061,1.0,0.000061,0.000058,F,0.000059,0.0,Major,0.000059,0.001008,4/4,0.000061,0.769919,https://i.scdn.co/image/ab67616d00001e02e42f55...
109571,Pop,Gunna,Out The Hood,3OGGzDmPU14szwANX5tbbn,72,0.000065,0.000068,1.0,0.000065,0.000063,F,0.000063,0.0,Minor,0.000065,0.001012,4/4,0.000066,0.769919,https://i.scdn.co/image/ab67616d00001e02bddf00...
149387,Pop,R3HAB,Fuego,1V4KGRCZuPMEBUVnJMivVx,70,0.000051,0.000055,1.0,0.000056,0.000051,F,0.000054,0.0,Minor,0.000052,0.000989,4/4,0.000055,0.769919,https://i.scdn.co/image/ab67616d00001e0251e1b1...
109733,Pop,Aminé,SHINE,1OWGLpptXlHLw1yibeHiHa,69,0.000055,0.000058,1.0,0.000057,0.000053,F,0.000054,0.0,Minor,0.000055,0.000986,4/4,0.000056,0.769919,https://i.scdn.co/image/ab67616d00001e02a52fb6...
110061,Pop,blackbear,4u,1XcXhu1X0xBFMDFyq0b7Ue,67,0.000045,0.000048,1.0,0.000046,0.000043,F,0.000043,0.0,Major,0.000043,0.001013,4/4,0.000048,0.769919,https://i.scdn.co/image/ab67616d00001e02bf33fd...
152092,Pop,Sody,Maybe It Was Me,0dlTFvu6sfzEenCh2IAPmu,63,0.000058,0.000057,1.0,0.000056,0.000054,F,0.000055,0.0,Major,0.000055,0.000983,4/4,0.000056,0.769919,https://i.scdn.co/image/ab67616d00001e02cfb6b7...
149662,Pop,Playboi Carti,Yah Mean,5MUxrNd7Gr2HksLcAlB0IO,64,0.000039,0.000043,1.0,0.000043,0.000038,F,0.000039,0.0,Minor,0.000040,0.001022,4/4,0.000043,0.769919,https://i.scdn.co/image/ab67616d00001e02e31a27...
152109,Pop,Aminé,SUGARPARENTS (feat. Rico Nasty),2WS2N8wgs9Drd5MNJLJxkf,58,0.000054,0.000061,1.0,0.000058,0.000053,F,0.000054,0.0,Major,0.000054,0.001062,4/4,0.000059,0.769919,https://i.scdn.co/image/ab67616d00001e02a52fb6...
110173,Pop,Shawn Mendes,When You're Ready,6bNcbh4SxQDU0BHa4Dj3wx,70,0.000057,0.000055,1.0,0.000055,0.000052,F,0.000053,0.0,Major,0.000053,0.001064,4/4,0.000056,0.769919,https://i.scdn.co/image/ab67616d00001e02269423...
